<a href="https://colab.research.google.com/github/proywm/PDC-concepts-LiveCoding/blob/main/LiveCoding_Concurrency_Simulating_Transportation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib

In [14]:
import time
from IPython.display import clear_output

# Setup
total_passengers = 10
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []

# Track the position of the car
car_position = 0

def print_smileys(count, total_slots=10):
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(2):  # Always print a 2x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane):
    global total_trips, passengers_transported, car_position
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            if passengers_transported >= total_passengers:
                return
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        if passengers_transported < total_passengers:
            total_trips += 1
            passengers_transported += bus_capacity
            if passengers_transported > total_passengers:
                passengers_transported = total_passengers
            trip_details.append((total_trips, passengers_transported))
            car_position = len(bus_paths) - 1
            visualize_trip(total_trips, passengers_transported, final=True)
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported, returning=True)
            time.sleep(0.25)
        time.sleep(1)


# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start)
    stop_smileys = print_smileys(num_smileys_stop)

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]

    car_pos = car_path[car_position]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(2):
        if row == 0:
            print(start_smileys[row] + "  " + car_pos + "  " + stop_smileys[row])
        else:
            print(start_smileys[row] + "  " + " " * len(car_pos) + "  " + stop_smileys[row])

# Run the transport function
start_time = time.time()
transport_passengers("🚗", 0)
elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜                                   😃😃😃😃😃
Total elapsed time: 32.71 seconds
Throughput: 0.31 passengers per second



In [15]:
import time
import threading
from IPython.display import clear_output

# Setup
total_passengers = 10
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []
lock = threading.Lock()

# Track the positions of each car
car_positions = {"🚗": 0, "🚙": 0}

def print_smileys(count, total_slots=10):
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(2):  # Always print a 2x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_id, car_emoji, lane, start_delay=0):
    global total_trips, passengers_transported
    time.sleep(start_delay)  # Start delay for the second car
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            with lock:
                if passengers_transported >= total_passengers:
                    return
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        with lock:
            if passengers_transported < total_passengers:
                total_trips += 1
                passengers_transported += bus_capacity
                if passengers_transported > total_passengers:
                    passengers_transported = total_passengers
                trip_details.append((total_trips, passengers_transported))
                car_positions[car_emoji] = len(bus_paths) - 1
                visualize_trip(total_trips, passengers_transported, final=True)
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            with lock:
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported, returning=True)
            time.sleep(0.25)
        time.sleep(1)

# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start)
    stop_smileys = print_smileys(num_smileys_stop)

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car1_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]
    car2_path = ["🚙" + fixed_path_segment * 3, fixed_path_segment + "🚙" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚙" + fixed_path_segment, fixed_path_segment * 3 + "🚙"]

    car1_pos = car1_path[car_positions["🚗"]]
    car2_pos = car2_path[car_positions["🚙"]]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(2):
        if row == 0:
            print(start_smileys[row] + "  " + car1_pos + "  " + stop_smileys[row])
        else:
            print(start_smileys[row] + "  " + car2_pos + "  " + stop_smileys[row])

# Create and start threads for two cars in different lanes
car1 = threading.Thread(target=transport_passengers, args=(1, "🚗", 0))
car2 = threading.Thread(target=transport_passengers, args=(2, "🚙", 1))

start_time = time.time()
car1.start()
car2.start()

car1.join()
car2.join()

elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜  🚙------------------------------  😃😃😃😃😃
Total elapsed time: 16.38 seconds
Throughput: 0.61 passengers per second



In [22]:
import time
import threading
from IPython.display import clear_output

# Setup
total_passengers = 10  # Adjusted for more passengers since we have 3 cars
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []
lock = threading.Lock()

# Track the positions of each car
car_positions = {"🚗": 0, "🚙": 0, "🚐": 0}

def print_smileys(count, total_slots=15):  # Adjusted for 15 passengers to match the grid
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(3):  # Adjusted for a 3x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane, start_delay=0):
    global total_trips, passengers_transported
    time.sleep(start_delay)  # Start delay for subsequent cars
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment * 3}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment * 2}",
        f"{fixed_path_segment * 2}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment * 3}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            with lock:
                if passengers_transported >= total_passengers:
                    return
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        with lock:
            if passengers_transported < total_passengers:
                total_trips += 1
                passengers_transported += bus_capacity
                if passengers_transported > total_passengers:
                    passengers_transported = total_passengers
                trip_details.append((total_trips, passengers_transported))
                car_positions[car_emoji] = len(bus_paths) - 1
                visualize_trip(total_trips, passengers_transported, final=True)
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            with lock:
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported, returning=True)
            time.sleep(0.25)
        time.sleep(1)

# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start, total_slots=15)  # Adjusted for more passengers
    stop_smileys = print_smileys(num_smileys_stop, total_slots=15)  # Adjusted for more passengers

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car1_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]
    car2_path = ["🚙" + fixed_path_segment * 3, fixed_path_segment + "🚙" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚙" + fixed_path_segment, fixed_path_segment * 3 + "🚙"]
    car3_path = ["🚐" + fixed_path_segment * 3, fixed_path_segment + "🚐" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚐" + fixed_path_segment, fixed_path_segment * 3 + "🚐"]

    car1_pos = car1_path[car_positions["🚗"]]
    car2_pos = car2_path[car_positions["🚙"]]
    car3_pos = car3_path[car_positions["🚐"]]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(3):
        if row == 0:
            print(start_smileys[row] + "  " + car1_pos + "  " + stop_smileys[row])
        elif row == 1:
            print(start_smileys[row] + "  " + car2_pos + "  " + stop_smileys[row])
        elif row == 2:
            print(start_smileys[row] + "  " + car3_pos + "  " + stop_smileys[row])

# Create and start threads for three cars in different lanes
car1 = threading.Thread(target=transport_passengers, args=("🚗", 0, 0))
car2 = threading.Thread(target=transport_passengers, args=("🚙", 1, 0.5))
car3 = threading.Thread(target=transport_passengers, args=("🚐", 2, 1))

start_time = time.time()
car1.start()
car2.start()
car3.start()

car1.join()
car2.join()
car3.join()

elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜  --------------------🚙----------  😃😃😃😃😃
⬜⬜⬜⬜⬜  🚐------------------------------  ⬜⬜⬜⬜⬜
Total elapsed time: 13.11 seconds
Throughput: 0.76 passengers per second



In [10]:
import time
from IPython.display import clear_output

# Setup
total_passengers = 10
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []

# Track the position of the car
car_position = 0

# Track waiting start times of passengers
waiting_start_times = [[None for _ in range(5)] for _ in range(3)]  # 3x5 matrix with initial None
boarding_times = []

def initialize_waiting_times():
    current_time = time.time()
    for row in range(3):
        for col in range(5):
            waiting_start_times[row][col] = current_time

def print_smileys(count, total_slots=10):
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(2):  # Always print a 2x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane):
    global total_trips, passengers_transported, car_position
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment}{fixed_path_segment}{fixed_path_segment}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            if passengers_transported >= total_passengers:
                return
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported)
            time.sleep(0.25)
        if passengers_transported < total_passengers:
            total_trips += 1
            row, col = divmod(passengers_transported, 5)  # Determine the passenger position in the matrix
            passengers_transported += bus_capacity
            if passengers_transported > total_passengers:
                passengers_transported = total_passengers
            trip_details.append((total_trips, passengers_transported))
            car_position = len(bus_paths) - 1
            visualize_trip(total_trips, passengers_transported, final=True)
            boarding_times.append((time.time(), row, col))

        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            car_position = bus_paths.index(path)
            visualize_trip(total_trips, passengers_transported, returning=True)

            time.sleep(0.25)
        time.sleep(1)

# Calculate and print average waiting time
def print_average_waiting_time():
    total_waiting_time = 0
    count = len(boarding_times)
    for boarding_time, row, col in boarding_times:
        if waiting_start_times[row][col] is not None:
            total_waiting_time += boarding_time - waiting_start_times[row][col]
    average_waiting_time = total_waiting_time / count if count > 0 else 0
    print(f"Average waiting time: {average_waiting_time:.2f} seconds")

# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start)
    stop_smileys = print_smileys(num_smileys_stop)

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]

    car_pos = car_path[car_position]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(2):
        if row == 0:
            print(start_smileys[row] + "  " + car_pos + "  " + stop_smileys[row])
        else:
            print(start_smileys[row] + "  " + " " * len(car_pos) + "  " + stop_smileys[row])

# Run the transport function
initialize_waiting_times()
start_time = time.time()
transport_passengers("🚗", 0)
elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")
throughput = passengers_transported / elapsed_time
print(f"Throughput: {throughput:.2f} passengers per second")
print_average_waiting_time()
print()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜                                   😃😃😃😃😃
Total elapsed time: 32.70 seconds
Throughput: 0.31 passengers per second
Average waiting time: 15.72 seconds



In [9]:
import time
import threading
from IPython.display import clear_output

# Setup
total_passengers = 10  # Adjusted for more passengers since we have 3 cars
bus_capacity = 1
total_trips = 0
passengers_transported = 0
trip_details = []
lock = threading.Lock()

# Track the positions of each car
car_positions = {"🚗": 0, "🚙": 0, "🚐": 0}

# Track waiting start times of passengers
waiting_start_times = [[None for _ in range(5)] for _ in range(3)]  # 3x5 matrix with initial None
boarding_times = []

def initialize_waiting_times():
    current_time = time.time()
    for row in range(3):
        for col in range(5):
            waiting_start_times[row][col] = current_time

def print_smileys(count, total_slots=15):  # Adjusted for 15 passengers to match the grid
    smileys = "😃" * count + "⬜" * (total_slots - count)
    smiley_grid = []
    for i in range(3):  # Adjusted for a 3x5 grid
        smiley_grid.append(smileys[i*5:(i+1)*5])
    return smiley_grid

# Transport function for a single car
def transport_passengers(car_emoji, lane, start_delay=0):
    global total_trips, passengers_transported
    time.sleep(start_delay)  # Start delay for subsequent cars
    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    bus_paths = [
        f"{car_emoji}{fixed_path_segment * 3}",
        f"{fixed_path_segment}{car_emoji}{fixed_path_segment * 2}",
        f"{fixed_path_segment * 2}{car_emoji}{fixed_path_segment}",
        f"{fixed_path_segment * 3}{car_emoji}"
    ]

    while True:
        for path in bus_paths:
            with lock:
                if passengers_transported >= total_passengers:
                    return
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported)

            time.sleep(0.25)
        with lock:
            if passengers_transported < total_passengers:
                total_trips += 1
                row, col = divmod(passengers_transported, 5)  # Determine the passenger position in the matrix
                passengers_transported += bus_capacity
                if passengers_transported > total_passengers:
                    passengers_transported = total_passengers
                trip_details.append((total_trips, passengers_transported))
                car_positions[car_emoji] = len(bus_paths) - 1
                visualize_trip(total_trips, passengers_transported, final=True)
                boarding_times.append((time.time(), row, col))
        time.sleep(0.25)  # Simulate the return of the bus
        for path in reversed(bus_paths):
            with lock:
                car_positions[car_emoji] = bus_paths.index(path)
                visualize_trip(total_trips, passengers_transported, returning=True)

            time.sleep(0.25)
        time.sleep(1)

# Calculate and print average waiting time
def print_average_waiting_time():
    total_waiting_time = 0
    count = len(boarding_times)
    for boarding_time, row, col in boarding_times:
        if waiting_start_times[row][col] is not None:
            total_waiting_time += boarding_time - waiting_start_times[row][col]
    average_waiting_time = total_waiting_time / count if count > 0 else 0
    print(f"Average waiting time: {average_waiting_time:.2f} seconds")


# Visualization
def visualize_trip(trip_number, passengers_transported, final=False, returning=False):
    clear_output(wait=True)
    num_smileys_start = total_passengers - passengers_transported
    num_smileys_stop = passengers_transported
    start_smileys = print_smileys(num_smileys_start, total_slots=15)  # Adjusted for more passengers
    stop_smileys = print_smileys(num_smileys_stop, total_slots=15)  # Adjusted for more passengers

    fixed_path_segment = "-" * 10  # Each segment between start, intermediate, and stop
    car1_path = ["🚗" + fixed_path_segment * 3, fixed_path_segment + "🚗" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚗" + fixed_path_segment, fixed_path_segment * 3 + "🚗"]
    car2_path = ["🚙" + fixed_path_segment * 3, fixed_path_segment + "🚙" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚙" + fixed_path_segment, fixed_path_segment * 3 + "🚙"]
    car3_path = ["🚐" + fixed_path_segment * 3, fixed_path_segment + "🚐" + fixed_path_segment * 2,
                 fixed_path_segment * 2 + "🚐" + fixed_path_segment, fixed_path_segment * 3 + "🚐"]

    car1_pos = car1_path[car_positions["🚗"]]
    car2_pos = car2_path[car_positions["🚙"]]
    car3_pos = car3_path[car_positions["🚐"]]

    print(f"Trip {trip_number}: {passengers_transported} passengers transported.")
    for row in range(3):
        if row == 0:
            print(start_smileys[row] + "  " + car1_pos + "  " + stop_smileys[row])
        elif row == 1:
            print(start_smileys[row] + "  " + car2_pos + "  " + stop_smileys[row])
        elif row == 2:
            print(start_smileys[row] + "  " + car3_pos + "  " + stop_smileys[row])
    elapsed_time = time.time() - start_time
    throughput = passengers_transported / elapsed_time
    print(f"Throughput: {throughput:.2f} passengers per second")
    print()

# Create and start threads for three cars in different lanes
def start_simulation():
    initialize_waiting_times()
    car1 = threading.Thread(target=transport_passengers, args=("🚗", 0, 0))
    car2 = threading.Thread(target=transport_passengers, args=("🚙", 1, 0.5))
    car3 = threading.Thread(target=transport_passengers, args=("🚐", 2, 1))

    global start_time
    start_time = time.time()
    car1.start()
    car2.start()
    car3.start()

    car1.join()
    car2.join()
    car3.join()

    elapsed_time = time.time() - start_time
    print(f"Total elapsed time: {elapsed_time:.2f} seconds")
    throughput = passengers_transported / elapsed_time
    print(f"Throughput: {throughput:.2f} passengers per second")
    print_average_waiting_time()
    print()

# Start the simulation
start_simulation()


Trip 10: 10 passengers transported.
⬜⬜⬜⬜⬜  🚗------------------------------  😃😃😃😃😃
⬜⬜⬜⬜⬜  --------------------🚙----------  😃😃😃😃😃
⬜⬜⬜⬜⬜  🚐------------------------------  ⬜⬜⬜⬜⬜
Throughput: 0.84 passengers per second

Total elapsed time: 13.10 seconds
Throughput: 0.76 passengers per second
Average waiting time: 5.39 seconds



In [40]:
import time
import threading
import random
from IPython.display import clear_output

# Setup
total_passengers = 200  # Number of passengers
counter = 0
lock = threading.Lock()

# Function that increments the counter
def increment_counter(passenger_id):
    global counter
    # Uncomment the next line to introduce a lock and prevent race condition
    #with lock:
    # Simulate some work with random delay
    local_counter = counter
    time.sleep(random.uniform(0.001, 0.01))  # Artificial random delay to increase the chance of race condition
    local_counter += 1
    counter = local_counter
    visualize(passenger_id)

# Visualization function
def visualize(passenger_id):
    clear_output(wait=True)
    print(f"Passenger {passenger_id} incremented the counter.")
    print(f"Current counter value: {counter}")
    print("Passengers who have incremented the counter:")
    for i in range(total_passengers):
        if i <= passenger_id:
            print("😃", end=" ")
        else:
            print("⬜", end=" ")
    print()

# Function that simulates a passenger incrementing the counter
def passenger_thread(passenger_id):
    time.sleep(0.05 * passenger_id)  # Stagger the passengers a bit
    increment_counter(passenger_id)

# Create and start threads for each passenger
threads = []
for i in range(total_passengers):
    t = threading.Thread(target=passenger_thread, args=(i,))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

print("All passengers have incremented the counter.")
print(f"Final counter value: {counter}")


Passenger 199 incremented the counter.
Current counter value: 200
Passengers who have incremented the counter:
😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 😃 
All passengers have incremented the counter.
Final counter value: 200
